# PREPROCESSING

- [Importar librerías](#Importar-librerías)
- [Lectura de los datasets](#Lectura-de-los-datasets)
- [Eliminación de características irrelevantes](#Eliminación-de-características-irrelevantes)
- [Manejo de datos faltantes](#Manejo-de-datos-faltantes)
- [Manejo de outliers](#Manejo-de-outliers)
- [Gestión de tipos](Gestión-de-tipos)
- [Codificación-de-variables-categóricas](Codificación-de-variables-categóricas)
- [Normalización y estandarización](#Normalización-y-estandarización)
- [Transformaciones de datos](#Transformaciones-de-datos)

## Importar librerías

!pip install -r requirements.txt

In [13]:
import os
import shutil
import zipfile
import pandas as pd
import numpy as np

## Lectura de los datasets

In [14]:
INPUT_ZIP = "./00_Data/Raw/titanic.zip"  # Directorio del zip
OUTPUT_FOLDER = "./00_Data/Clean"  # Directorio de destino
TRAIN_FILENAME = "train.csv"  # Nombre del fichero de entrenamiento
TEST_FILENAME = "test.csv"  # Nombre del fichero de entrenamiento

def fetch_data(input_path=INPUT_ZIP, output_dir=OUTPUT_FOLDER):
    """
    Extrae el contenido de un archivo ZIP en un directorio de destino.

    Parámetros:
    -----------
    input_path : str, opcional
        Ruta al archivo ZIP que se desea descomprimir. El valor predeterminado es la variable 'INPUT_ZIP'.
        
    output_dir : str, opcional
        Directorio en el cual se extraerá el contenido del archivo ZIP. Si el directorio no existe,
        será creado automáticamente. El valor predeterminado es la variable 'OUTPUT_FOLDER'.

    Comportamiento:
    ---------------
    - Crea el directorio de destino si no existe.
    - Descomprime el archivo ZIP en el directorio de destino.

    Excepciones:
    ------------
    Puede lanzar una excepción si el archivo ZIP no existe o si hay problemas al descomprimirlo.

    Ejemplo de uso:
    ---------------
    fetch_data('data.zip', 'output/')
    """
    # Comprobación de que el directorio de destino existe
    os.makedirs(output_dir, exist_ok=True)

    # Descomprime el archivo ZIP en caso de que no haya ningún csv en la carpeta
    if(len([file for file in os.listdir(output_dir) if file.endswith('.csv')]) == 0):
        with zipfile.ZipFile(input_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)


def load_data(directory=OUTPUT_FOLDER, filename=TRAIN_FILENAME):
    """
    Lee un archivo CSV desde el directorio especificado.

    Parámetros:
    -----------
    directory : str
        El directorio donde se encuentra el archivo CSV.
        
    filename : str
        El nombre del archivo CSV a leer (incluyendo la extensión .csv).

    Retorna:
    --------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos del archivo CSV.

    Excepciones:
    ------------
    FileNotFoundError:
        Se lanza si el archivo no existe en el directorio dado.
    
    Ejemplo de uso:
    ---------------
    df = read_csv_from_directory('data', 'file.csv')
    """
    # Construir la ruta completa al archivo CSV
    file_path = os.path.join(directory, filename)

    # Verificar si el archivo existe
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"El archivo {filename} no se encuentra en el directorio {directory}")

    # Leer el archivo CSV en un DataFrame
    return pd.read_csv(file_path)

fetch_data()
df_train = load_data(OUTPUT_FOLDER, TRAIN_FILENAME)
df_test = load_data(OUTPUT_FOLDER, TEST_FILENAME)

print("Train dataset:", df_train.shape)
print("Test dataset:", df_test.shape)

Train dataset: (227845, 31)
Test dataset: (56962, 31)


## Eliminación de características irrelevantes

In [15]:
# Seleccionamos las columnas con las que nos queremos quedar
cols_keep = ["V2", "V4", "V8", "V11", "V19", "V20", "V21", "V27", "V28"]
predict_col = "Class"


In [16]:
print("Las columnas originales son:", df_train.columns.tolist())

df_train = df_train[cols_keep + [predict_col]]
df_test = df_test[cols_keep]

print("Las columnas tras eliminar irrelevantes son:", df_train.columns.tolist())

Las columnas originales son: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
Las columnas tras eliminar irrelevantes son: ['V2', 'V4', 'V8', 'V11', 'V19', 'V20', 'V21', 'V27', 'V28', 'Class']


## Manejo de datos faltantes

In [17]:
df_train.isnull().sum()

V2       0
V4       0
V8       0
V11      0
V19      0
V20      0
V21      0
V27      0
V28      0
Class    0
dtype: int64

In [18]:
df_test.isnull().sum()

V2     0
V4     0
V8     0
V11    0
V19    0
V20    0
V21    0
V27    0
V28    0
dtype: int64

## Manejo de outliers

In [19]:

# Columnas a analizar (excluyendo Class)
cols_excluir = ["Class"]  
cols_outlier = [col for col in df_train.columns if col not in cols_excluir]

# Calcular límites de outliers en train
outlier_limits = {}
for col in cols_outlier:
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    IQR = Q3 - Q1
    outlier_limits[col] = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)

# Ver cuántos outliers hay en cada columna
outliers_train = {col: df_train[(df_train[col] < outlier_limits[col][0]) | (df_train[col] > outlier_limits[col][1])][col].count() for col in cols_outlier}

print("Cantidad de outliers en df_train por columna:")
print(outliers_train)

Cantidad de outliers en df_train por columna:
{'V2': np.int64(10824), 'V4': np.int64(8910), 'V8': np.int64(19249), 'V11': np.int64(622), 'V19': np.int64(8206), 'V20': np.int64(22135), 'V21': np.int64(11548), 'V27': np.int64(31220), 'V28': np.int64(24182)}


In [20]:
#windor
df_train_winsorized = df_train.copy()
for col in cols_outlier:
    lower, upper = outlier_limits[col]
    df_train_winsorized[col] = np.clip(df_train_winsorized[col], lower, upper)

print(df_train_winsorized)

              V2        V4        V8       V11       V19       V20       V21  \
0      -0.380783  0.330155  0.035994  0.624996  0.167987 -0.125390  0.238197   
1      -0.626943 -2.017772 -0.401619 -1.945070 -1.692780 -0.470372 -0.153485   
2       0.820566 -0.709897  0.117276  0.521931  0.198772  0.012227 -0.314638   
3       1.014587  2.769390 -0.081323 -0.855437 -1.826983 -0.253757  0.063525   
4       1.897371 -0.029571  1.131406  0.472626  0.712695 -0.012320 -0.480691   
...          ...       ...       ...       ...       ...       ...       ...   
227840  0.100792  0.461596  0.119221  1.585195 -0.182494 -0.157534 -0.186027   
227841  0.144023 -1.241113  1.131406 -0.421422  0.094703 -0.295730  0.037078   
227842  0.992946  0.485774 -0.395316 -0.382362 -0.229614  0.007155  0.052649   
227843  2.354849 -3.235620  1.131406 -0.418939 -1.143752  0.417396 -0.332759   
227844 -0.784851 -0.698559 -0.188057 -0.546288  0.673568  0.337732  0.027634   

             V27       V28  Class  
0  

In [21]:
# Columnas a analizar (Class)
cols_excluir = ["Class"]  
cols_outlier = [col for col in df_train_winsorized.columns if col not in cols_excluir]

# Calcular límites de outliers en train
outlier_limits = {}
for col in cols_outlier:
    Q1 = df_train_winsorized[col].quantile(0.25)
    Q3 = df_train_winsorized[col].quantile(0.75)
    IQR = Q3 - Q1
    outlier_limits[col] = (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)

# Ver cuántos outliers hay en cada columna
outliers_train = {col: df_train_winsorized[(df_train_winsorized[col] < outlier_limits[col][0]) | (df_train_winsorized[col] > outlier_limits[col][1])][col].count() for col in cols_outlier}

print("Cantidad de outliers en df_train_scaled por columna:")
print(outliers_train)

Cantidad de outliers en df_train_scaled por columna:
{'V2': np.int64(0), 'V4': np.int64(0), 'V8': np.int64(0), 'V11': np.int64(0), 'V19': np.int64(0), 'V20': np.int64(0), 'V21': np.int64(0), 'V27': np.int64(0), 'V28': np.int64(0)}


In [22]:
df_train.describe()

,V2,V4,V8,V11,V19,V20,V21,V27,V28,Class
count,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000
mean,-0.000384,-0.001458,-0.000383,0.000577,0.000113,0.000398,0.000215,-0.000032,0.000257,0.001729
std,1.656602,1.415853,1.188643,1.021130,0.814306,0.774045,0.732617,0.401821,0.329776,0.041548
min,-72.715728,-5.683171,-73.216718,-4.797473,-7.213527,-54.497720,-34.830382,-9.895244,-15.430084,0.000000
25%,-0.597971,-0.849270,-0.208431,-0.763311,-0.455756,-0.211969,-0.228730,-0.070960,-0.052980,0.000000
50%,0.066050,-0.020959,0.022233,-0.031764,0.004464,-0.062614,-0.029639,0.001359,0.011366,0.000000
75%,0.803898,0.741630,0.327504,0.740126,0.458395,0.133017,0.186080,0.091400,0.078464,0.000000
max,22.057729,16.875344,20.007208,12.018913,5.591971,39.420904,27.202839,31.612198,33.847808,1.000000


In [23]:
df_train_winsorized.describe()

,V2,V4,V8,V11,V19,V20,V21,V27,V28,Class
count,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000,227845.000000
mean,0.050431,-0.029360,0.068464,-0.002777,-0.003917,-0.028364,-0.015331,0.012624,0.011246,0.001729
std,1.121805,1.310303,0.485234,1.003332,0.759039,0.317885,0.323893,0.165498,0.124361,0.041548
min,-2.700774,-3.235620,-1.012333,-3.018468,-1.826983,-0.729448,-0.850945,-0.314500,-0.250147,0.000000
25%,-0.597971,-0.849270,-0.208431,-0.763311,-0.455756,-0.211969,-0.228730,-0.070960,-0.052980,0.000000
50%,0.066050,-0.020959,0.022233,-0.031764,0.004464,-0.062614,-0.029639,0.001359,0.011366,0.000000
75%,0.803898,0.741630,0.327504,0.740126,0.458395,0.133017,0.186080,0.091400,0.078464,0.000000
max,2.906702,3.127981,1.131406,2.995283,1.829621,0.650495,0.808295,0.334940,0.275631,1.000000


## Escritura de los dataframes resultantes

In [24]:
OUTPUT_FOLDER = "./00_Data/Cleaned/"

def save_dataframes_to_csv(output_folder, df_train, df_test, train_filename="train_clean.csv", test_filename="test_clean.csv"):
    """
    Guarda los DataFrames de entrenamiento y prueba en formato CSV en una carpeta específica.
    Si la carpeta ya existe, borra todo su contenido antes de guardar los nuevos archivos.
    
    Args:
        output_folder (str): La ruta de la carpeta donde se guardarán los archivos CSV.
        df_train (pd.DataFrame): El DataFrame de entrenamiento que se va a guardar.
        df_test (pd.DataFrame): El DataFrame de prueba que se va a guardar.
        train_filename (str, opcional): El nombre del archivo CSV para el DataFrame de entrenamiento.
        test_filename (str, opcional): El nombre del archivo CSV para el DataFrame de prueba.
    
    """
    # Si la carpeta ya existe, eliminar todo su contenido
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)  # Borrar toda la carpeta y su contenido
        print(f"Carpeta {output_folder} eliminada.")
    
    # Crear la carpeta si no existe
    os.makedirs(output_folder, exist_ok=True)
    
    # Definir las rutas completas de los archivos
    train_path = os.path.join(output_folder, train_filename)
    test_path = os.path.join(output_folder, test_filename)
    
    # Guardar los DataFrames en formato CSV
    df_train.to_csv(train_path, index=False)
    df_test.to_csv(test_path, index=False)
    
    print(f"DataFrames guardados en {output_folder}:")
    print(f" - {train_filename}")
    print(f" - {test_filename}")

save_dataframes_to_csv(OUTPUT_FOLDER, df_train, df_test)

Carpeta ./00_Data/Cleaned/ eliminada.
DataFrames guardados en ./00_Data/Cleaned/:
 - train_clean.csv
 - test_clean.csv
